# Course project: cleavage plane for Ca doped YBCO

In [27]:
import numpy as np, os
import matplotlib.pyplot as plt
from pwscf import write_pwscf_input
from labutil.plugins.pwscf import run_qe_pwscf, PWscf_inparam, parse_qe_pwscf_output
from labutil.objects import Struc, Dir, ase2struc, Kpoints, PseudoPotential
from ase.spacegroup import crystal
from ase.io import write
from ase.build import bulk, make_supercell, add_vacuum
from ase import Atoms
import numpy, os, copy
from labutil.objects import TextFile, ExternalCode, File, Param
from labutil.util import prepare_dir, run_command

Make a structure from standard cell lattice parameters.

In [46]:
def make_struc_non_doped(nxy=1, nz = 2, alat=3.82, blat=3.89, clat=11.68, vacuum=0, cleave_plane='NO',
                         separation=0):
    """
    Creates the crystal structure using ASE and saves to a cif file. Constructs a root2xroot2 YBCO structure
    nxy, nz: unit cell dimensions follow  nxy *root 2, nxy* root 2, nz 
    alat, blat, clat: conventianal (NOT root2) lattice parameters
    vacuum: vacuum spacing between slabs
    cleave_plane: Not yet implemented
    separation: not yet implemented
    :return: structure object converted from ase
    """
    a = np.sqrt(alat**2 + blat**2)
    lattice = np.array([[a,0,0],[0,a,0],[0,0,clat]])
    symbols = ['Y', 'Y', 'Ba', 'Ba', 'Ba', 'Ba', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu', 'Cu','O', 'O', 'O','O','O','O','O','O','O','O','O','O','O','O',]
    sc_pos = [[0.5,0,0.5], [0,0.5,0.5], 
              [0.5, 0, 0.81939], [0, 0.5, 0.81939], [0.5, 0, 0.18061], [0, 0.5, 0.18061],
              [0,0,0], [0.5,0.5,0], [0,0,0.64668], [0.5,0.5,0.64668], [0,0,0.35332], [0.5,0.5,0.35332],
              [0.25,0.25,0], [0.75,0.75,0], [0,0,0.15918], [0.5,0.5,0.15918], [0.25,0.25,0.37835], [0.75,0.75,0.37835],[0.25,0.75,0.37935], [0.75,0.25,0.37935], [0.25,0.25,0.62065], [0.75,0.75,0.62065],[0.25,0.75,0.62165], [0.75,0.25,0.62165], [0,0,0.84082], [0.5,0.5,0.84082]
             ]
    YBCO = Atoms(symbols=symbols, scaled_positions=sc_pos, cell=lattice)
    multiplier = np.identity(3)
    multiplier[0,0]=nxy
    multiplier[1,1]=nxy
    multiplier[2,2]=nz
    supercell = make_supercell(YBCO, multiplier)
    add_vacuum(supercell, vacuum)
    write(f'YBCO_{nxy}{nxy}{nz}_{vacuum}vac_{cleave_plane}cleave_{separation}sep.cif', supercell)
    structure = Struc(ase2struc(supercell))
    return structure

In [49]:
struc = make_struc_non_doped(nxy = 1, nz = 3, vacuum = 40)

write an input file

In [47]:
def write_inputs(ecut = 80, nkxy = 8, nkz = 1, ucs = 3, struc = None, dirname = 'Test', name = 'YBCO', calc = 'relax'):
    pseudopots = {'Y': PseudoPotential(ptype='uspp', element='Y', functional='LDA', name='Y.pz-spn-rrkjus_psl.1.0.0.UPF'),
                  'Ba': PseudoPotential(ptype='uspp', element='Ba', functional='LDA', name='Ba.pz-spn-rrkjus_psl.1.0.0.UPF'),
                  'Cu': PseudoPotential(ptype='uspp', element='Cu', functional='LDA', name='Cu.pz-d-rrkjus.UPF'),
                  'O': PseudoPotential(ptype='uspp', element='O', functional='LDA', name='O.pz-rrkjus.UPF')}
    kpts = Kpoints(gridsize=[nkxy, nkxy, nkz], option='automatic', offset=True)
    runpath = Dir(path=os.path.join(os.environ['WORKDIR'], 'AP275', dirname))
    input_params = PWscf_inparam({
        'CONTROL': {
            'calculation': calc,
            'pseudo_dir': os.environ['QE_POTENTIALS'],
            'outdir': runpath.path,
            'tstress': True,
            'tprnfor': True,
            'disk_io': 'none'
        },
        'SYSTEM': {
            'ecutwfc': ecut,
            'ecutrho': ecut * 12,
            'occupations': 'smearing',
            'smearing': 'mp',
            'degauss': 0.02
             },
        'ELECTRONS': {
            'diagonalization': 'david',
            'mixing_beta': 0.7,
            'conv_thr': 1e-7,
        },
        'IONS': {
            'ion_dynamics': 'bfgs'
        },
        'CELL': {},
        })
        
    pwscf_code = ExternalCode({'path': os.environ['QE_PW_COMMAND']})
    prepare_dir(runpath.path)
    infile = write_pwscf_input(params=input_params, struc=struc, kpoints=kpts, runpath=runpath,
                               pseudopots=pseudopots, name = name, constraint=None)
    return infile

Inital relax calculation (RUNS A CALC)

In [ ]:
def relax_YBCO(nk, ecut):
    pseudopots = {'Y': PseudoPotential(ptype='uspp', element='Y', functional='LDA', name='Y.pz-spn-rrkjus_psl.1.0.0.UPF'),
                  'Ba': PseudoPotential(ptype='uspp', element='Ba', functional='LDA', name='Ba.pz-spn-rrkjus_psl.1.0.0.UPF'),
                  'Cu': PseudoPotential(ptype='uspp', element='Cu', functional='LDA', name='Cu.pz-d-rrkjus.UPF'),
                  'O': PseudoPotential(ptype='uspp', element='O', functional='LDA', name='O.pz-rrkjus.UPF')}
    struc = make_struc_non_doped()
    kpts = Kpoints(gridsize=[nk, nk, nk], option='automatic', offset=True)
    dirname = 'YBCO_relax_ecut_{}_nk_{}'.format(ecut, nk)
    runpath = Dir(path=os.path.join(os.environ['WORKDIR'], "Project", dirname))
    input_params = PWscf_inparam({
        'CONTROL': {
            'calculation': 'relax',
            'pseudo_dir': os.environ['QE_POTENTIALS'],
            'outdir': runpath.path,
            'tstress': True,
            'tprnfor': True,
            'disk_io': 'none'
        },
        'SYSTEM': {
            'ecutwfc': ecut,
            'ecutrho': ecut * 12,
            'occupations': 'smearing',
            'smearing': 'mp',
            'degauss': 0.02
             },
        'ELECTRONS': {
            'diagonalization': 'david',
            'mixing_beta': 0.7,
            'conv_thr': 1e-7,
        },
        'IONS': {
            'ion_dynamics': 'bfgs'
        },
        'CELL': {},
        })

    output_file = run_qe_pwscf(runpath=runpath, struc=struc,  pseudopots=pseudopots,
                               params=input_params, kpoints=kpts, ncpu=16)
    output = parse_qe_pwscf_output(outfile=output_file)
    return output

def kpts_scan():
    ecut = 30
    nk = np.arange(1, 2)
    energy=[]
    for i in nk:
        output = relax_YBCO(ecut=ecut, nk=i)
        energy.append(output['energy'])
        print(i)
    return energy